In [9]:
# Import netket library
import netket as nk
from qiskit.chemistry import FermionicOperator
import itertools

# Helper libraries
import numpy as np
import os
import time
import json

# Own libraries
# import utility as ut

In [ ]:
%config Completer.use_jedi = False

In [198]:
nk.config.update('NETKET_EXPERIMENTAL', 1)

In [199]:
# Unpack data
# N = 2
# N = systemData['totalBasisSize']
# molecule = systemData['molecule']
# basis = systemData['basis']

# make filename
outfile = 'testH2'
nElectrons=2

#H2
OB = np.load('../data/integrals/STO-3G/STO-3G_H2_OB_d0-7339_eq1.npy')
TB = np.load('../data/integrals/STO-3G/STO-3G_H2_TB_d0-7339_eq1.npy')   

########
FerOp = FermionicOperator(OB, TB)


mapping = FerOp.mapping('jordan_wigner')
weights = [w[0] for w in mapping.paulis]
operators = [w[1].to_label() for w in mapping.paulis]

ha = nk.operator.PauliStrings(operators, weights)
hi = ha.hilbert

# seed
np.random.seed(seed=123)
nk.legacy.random.seed(seed=123)

# g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
# hi = nk.hilbert.Spin(s=0.5, N=g.n_nodes, total_sz=0) # Remove sector!!
# ha = nk.operator.
# ha = nk.operator.Heisenberg(graph=g, hilbert=hi)

ma = nk.models.RBM(alpha=2, use_visible_bias=True, use_hidden_bias=True, dtype=complex, kernel_init=nk.nn.initializers.normal(stddev=0.1))
# ma.init_random_parameters(seed=seed, sigma=0.1)

# sa = nk.sampler.MetropolisHamiltonian(graph=g, machine=ma)
# sa = nk.sampler.MetropolisExchange(hilbert=hi)
g = nk.graph.Hypercube(length=2, n_dim=1, pbc=False)
sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, n_chains=1)

In [208]:
ha.get_conn(np.array([0,1]))

(array([[0., 1.],
        [1., 0.]]),
 array([-1.12487186+0.j, -0.96643592+0.j]))

In [200]:
vs = nk.variational.MCState(sa, ma, n_samples=1000)
vs.init_parameters(nk.nn.initializers.normal(stddev=0.1))

In [201]:
sampleState = sa.reset(machine=ma, parameters=vs.parameters)
print(sampleState.σ)
nDown = len(np.array(sampleState.σ[0])) - np.array(sampleState.σ).sum()
print(nDown)

[[0. 1.]]
1.0


In [101]:
nElectrons/2

1.0

In [202]:
nDown = 0
while nDown != int(nElectrons/2): 
    sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, n_chains=1)
    vs = nk.variational.MCState(sa, ma, n_samples=1000)
    sampleState = vs.sampler_state
    nDown = len(np.array(sampleState.σ[0])) - np.array(sampleState.σ).sum()
    print(sampleState.σ, nDown)

[[0. 0.]] 2.0
[[1. 0.]] 1.0


In [203]:
opt = 'sgd'
vs.init_parameters(nk.nn.initializers.normal(stddev=0.1))
if opt == 'sgd':
    op = nk.optimizer.Sgd(learning_rate=0.01)
elif opt == 'adam':
    op = nk.optimizer.Adam(learning_rate=0.01)
else:
    raise NotImplementedError('optimizer not implemented: ', opt)

sr = nk.optimizer.SR(diag_shift=0.1) # Default 0.01

vmc = nk.VMC(
        hamiltonian=ha,
        optimizer=op,
        sr=sr,
        variational_state=vs)

print('the outfile is: ', outfile)
start = time.time()
vmc.run(out=outfile, n_iter=5000)
end = time.time()

  0%|          | 0/5000 [00:04<?, ?it/s]


ValueError: Expected tuple, got FrozenDict({
    Dense: {
        bias: Traced<ShapedArray(complex128[4])>with<DynamicJaxprTrace(level=0/1)>,
        kernel: Traced<ShapedArray(complex128[2,4])>with<DynamicJaxprTrace(level=0/1)>,
    },
    visible_bias: Traced<ShapedArray(complex128[2])>with<DynamicJaxprTrace(level=0/1)>,
}).

In [194]:
state = vs.sampler_state
state.σ

DeviceArray([[0., 1.]], dtype=float64)

In [157]:
sampleState.σ

DeviceArray([[0., 1.]], dtype=float64)